In [1]:
#!M
# !pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null 2>&1
# %pip install --upgrade pip
# %pip install -r /home/jupyter/work/resources/riiidNew/requirements.txt --upgrade
# print("ok")


In [ ]:
# "https://docs-python.ru/tutorial/operatsii-slovarjami-dict-python/metod-dict-update/"

In [2]:
#!M
import numpy as np
import pandas as pd
import pyarrow
from collections import defaultdict
import datatable as dt
import lightgbm as lgb

from catboost.utils import get_gpu_device_count
import catboost
from catboost import CatBoostClassifier, Pool

from matplotlib import pyplot as plt
from tqdm import tqdm
# import riiideducation
import torch
import pickle
import gc
from pathlib import Path

from bayes_opt import BayesianOptimization
from preprocess import preprocess_train_data  # , catboost_hyperparams
from torch import cuda



# Error handling, ignore all
np.seterr(divide = 'ignore', invalid = 'ignore')

print(f"pyarrow {pyarrow.__version__}")
print(f"curdir {Path.cwd()}")


pyarrow 2.0.0
curdir /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Challenges/riiidNew


In [3]:
#!M

# Data config

dtypes = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32', 
    'content_id': 'int16', 
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'answered_correctly': 'int8', 
    'prior_question_elapsed_time': 'float32', 
    'prior_question_had_explanation': 'bool',
    'user_answer': 'int8',
}

target = 'answered_correctly'

homedir = Path.home()
print(str(homedir))

if str(homedir) == "/home/sergey":   # Home computer
    kaggle_path = Path.cwd()/"kaggle_tmp/"
    questions_df = pd.read_csv('/mnt/data30G/2020riiid/questions.csv', usecols = [0, 3], dtype = {'question_id': 'int32', 'part': 'int32'})
    # train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv', columns = set(dtypes.keys())).to_pandas()
    # train_df = dt.fread('/mnt/data30G/2020riid/train.csv', columns = set(dtypes.keys())).to_pandas()
    test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train_1e5.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid_1e4.feather", columns=dtypes.keys())
    # train_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_train.feather", columns=dtypes.keys())
    # test_df = pd.read_feather("/home/sergey/mnt/4.5Tb/Downloads/riiidCVdata/cv1_valid.feather", columns=dtypes.keys())

elif str(homedir) == "/root":   # Datasphere
    
    path = Path.cwd()/"riiidNew"/"data"
    kaggle_path = Path.cwd()/"riiidNew"/"kaggle/"
    # questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int16', 'part': 'int8'})
    questions_df = pd.read_csv(path/'questions.csv', usecols = [0, 3], dtype = {'question_id': 'int32', 'part': 'int32'})
#     train_df = pd.read_feather(path/"cv1_train_1e5.feather", columns=dtypes.keys())
#     test_df = pd.read_feather(path/"cv1_valid_1e4.feather", columns=dtypes.keys())
    # test_df = pd.read_feather(path/"data/cv1_valid_1e4.feather", columns=dtypes.keys())
    # train_df = pd.read_feather(path/"cv1_train.feather", columns=dtypes.keys())
    # test_df = pd.read_feather(path/"cv1_valid.feather", columns=dtypes.keys())
    # train_df = dt.fread(path/'train.csv', columns=dtypes.keys()).to_pandas().astype(dtypes, errors="ignore")    
    train_df = pd.read_pickle(path/"cv1_train.pickle.zip").astype(dtypes, errors="ignore")
    # train_df = train_df.iloc[:int(1e5)]
    test_df = pd.read_pickle(path/"cv1_valid.pickle.zip").astype(dtypes, errors="ignore")
    # test_df = test_df.iloc[:int(1e4)]

print(f"train_df shape = {train_df.shape}")

/home/sergey
train_df shape = (100000, 10)


In [4]:
#!M
train_user_agg, train_content_agg, train_df = preprocess_train_data(train_df, questions_df, target, dtypes)
test_user_agg, test_content_agg, test_df = preprocess_train_data(test_df, questions_df, target, dtypes)

# Ratio is 6/24 = 25%
valid_df = train_df.groupby('user_id').tail(6)
train_df.drop(valid_df.index, inplace = True)

In [5]:
#!L
features = ['content_id', 'prior_question_elapsed_time', 
            'prior_question_had_explanation', 'user_correctness', 
            'part', 'content_count']

# Training and validating data
train_set = Pool(train_df[features], label = train_df[target])
val_set = Pool(valid_df[features], label = valid_df[target])
# val_set = Pool(test_df[features], label = test_df[target])

prior_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'custom_metric': 'AUC:hints=skip_train~false',
    'task_type': 'CPU',  #  'GPU' if cuda.is_available() else 'CPU',
    'grow_policy': 'Lossguide',
    'iterations': 2,
    'learning_rate': 4e-3,
    'random_seed': 0,
    'l2_leaf_reg': 4e-3,
    'depth': 6,
    'border_count': 128,
    'verbose': False, # 150,
    'od_type': 'Iter',
    'od_wait': 50,
    'max_leaves': 10,
    }


pds = {
    'l2_leaf_reg': [2e-1, 5e2],
    'depth': [6, 15],
    'max_leaves': [10, 120],
    'border_count': [50, 300],

}



In [6]:
def bayesian_catboost_searchCV(train_set, pds, init_points=3, n_iter=7):

    def catboost_hyperparams(**dict):
        params = {'loss_function': 'Logloss',
                  'eval_metric': 'AUC',
                  'custom_metric': 'AUC:hints=skip_train~false',
                  'task_type': 'CPU',  #  'GPU' if cuda.is_available() else 'CPU',
                  'grow_policy': 'Lossguide',
                  'iterations': 2,
                  'learning_rate': 4e-3,
                  'random_seed': 0,
                  'l2_leaf_reg': dict["l2_leaf_reg"],
                  'depth': int(dict["depth"]),
                  'border_count': int(dict['border_count']),
                  'verbose': False, # 150,
                  'od_type': 'Iter',
                  'od_wait': 50,
                  'max_leaves': int(dict["max_leaves"])
                }


        # Model definition
        model = CatBoostClassifier(**params)

        # Fitting
        scores = catboost.cv(train_set, params,
                             plot=False,
                             # metric_period=10,
                             type='TimeSeries',
                             fold_count=3)
        # model.fit(train_set, eval_set=val_set, use_best_model=True)

        # return np.max(model.get_best_score()["validation"]["AUC"])
        print("scores", scores)
        return np.max(scores["test-AUC-mean"])

    optimizer = BayesianOptimization(catboost_hyperparams, pds, random_state=0)
    optimizer.maximize(init_points, n_iter)
    return optimizer.max

In [7]:
print("ok")

ok


In [8]:
%%time
print("optimizer_maxCV")
optimizer_maxCV = bayesian_catboost_searchCV(train_set, pds=pds, init_points=5, n_iter=7)
optimizer_maxCV

|   iter    |  target   | border... |   depth   | l2_lea... | max_le... |
-------------------------------------------------------------------------
scores    iterations  test-AUC-mean  test-AUC-std  train-AUC-mean  train-AUC-std  \
0           0       0.763959      0.014742        0.756828       0.009946   
1           1       0.774311      0.010108        0.768959       0.006142   

   test-Logloss-mean  test-Logloss-std  train-Logloss-mean  train-Logloss-std  
0           0.692149          0.000186            0.692223           0.000132  
1           0.691095          0.000389            0.691238           0.000288  
|  1        |  0.7743   |  187.2    |  12.44    |  301.5    |  69.94    |
scores    iterations  test-AUC-mean  test-AUC-std  train-AUC-mean  train-AUC-std  \
0           0       0.771992      0.012131        0.766738       0.008736   
1           1       0.778075      0.007194        0.773278       0.004387   

   test-Logloss-mean  test-Logloss-std  train-Logloss-mean  

{'target': 0.7809165886816575,
 'params': {'border_count': 155.6946859118282,
  'depth': 9.689734696339546,
  'l2_leaf_reg': 224.03579335985572,
  'max_leaves': 104.00321450264359}}

In [9]:
exit(1)


{'loss_function': 'Logloss', 'eval_metric': 'AUC', 'task_type': 'GPU', 'grow_policy': 'Lossguide', 'iterations': 15000, 'learning_rate': 0.01, 'random_seed': 0, 'l2_leaf_reg': 0.1, 'depth': 8, 'border_count': 128, 'verbose': 150, 'od_type': 'Iter', 'od_wait': 50}
bestTest = 0.7371392846

bestTest = 0.7370638549
bestTest = 0.7364509702

In [12]:
def bayesian_catboost_search(train_set, val_set, prior_params, pds, init_points=3, n_iter=7):

    def catboost_hyperparams(**dict):
        # params = prior_params.update(dict)
        params = {'loss_function': 'Logloss',
                  'eval_metric': 'AUC',
                  'custom_metric': 'AUC:hints=skip_train~false',
                  'task_type': 'CPU',  #  'GPU' if cuda.is_available() else 'CPU',
                  'grow_policy': 'Lossguide',
                  'iterations': 2,
                  'learning_rate': 4e-3,
                  'random_seed': 0,
                  'l2_leaf_reg': dict["l2_leaf_reg"],
                  'depth': int(dict["depth"]),
                  'border_count': int(dict['border_count']),
                  'verbose': False, # 150,
                  'od_type': 'Iter',
                  'od_wait': 50,
                  'max_leaves': int(dict["max_leaves"])
                }


        # Model definition
        model = CatBoostClassifier(**params)

        # Fitting
        model.fit(train_set, eval_set=val_set, use_best_model=True)

        return np.max(model.get_best_score()["validation"]["AUC"])

    optimizer = BayesianOptimization(catboost_hyperparams, pds, random_state=0)
    optimizer.maximize(init_points, n_iter)
    return optimizer.max


In [13]:
optimzer_max = bayesian_catboost_search(train_set=train_set, val_set=val_set,
                                        prior_params=prior_params, pds=pds,
                                        init_points=5, n_iter=7)
optimzer_max

|   iter    |  target   | border... |   depth   | l2_lea... | max_le... |
-------------------------------------------------------------------------
|  1        |  0.7703   |  187.2    |  12.44    |  301.5    |  69.94    |
|  2        |  0.7696   |  155.9    |  11.81    |  218.9    |  108.1    |
|  3        |  0.7683   |  290.9    |  9.451    |  395.9    |  68.18    |
|  4        |  0.7705   |  185.4    |  6.406    |  290.6    |  65.7     |
|  5        |  0.7666   |  195.2    |  12.72    |  307.5    |  67.59    |
|  6        |  0.7656   |  203.7    |  13.71    |  417.9    |  78.25    |
|  7        |  0.7676   |  278.1    |  8.551    |  333.5    |  19.41    |
|  8        |  0.7653   |  261.5    |  12.68    |  172.4    |  72.84    |
|  9        |  0.7657   |  52.4     |  6.53     |  407.1    |  98.1     |
|  10       |  0.7664   |  196.8    |  10.08    |  458.0    |  65.63    |


{'target': 0.7705098101131143,
 'params': {'border_count': 185.37344797295015,
  'depth': 6.405537966108271,
  'l2_leaf_reg': 290.55373136202036,
  'max_leaves': 65.69970433836252}}

# Inference

In [ ]:
#!M
user_sum_dict = train_user_agg['sum'].astype('int32').to_dict(defaultdict(int))
user_count_dict = train_user_agg['count'].astype('int32').to_dict(defaultdict(int))
content_sum_dict = train_content_agg['sum'].astype('int32').to_dict(defaultdict(int))
content_count_dict = train_content_agg['count'].astype('int32').to_dict(defaultdict(int))

for filename, dic in zip(["user_sum_dict", "user_count_dict", "content_sum_dict", "content_count_dict"],
                         [user_sum_dict, user_count_dict, content_sum_dict, content_count_dict]):
    with open(f'{kaggle_path}/{filename}.pickle', 'wb') as handle:
        pickle.dump(dic, handle)


In [ ]:
train_df.head()
del train_df
gc.collect()


In [ ]:
%%time
validaten_flg = True
if validaten_flg:
    from emulator import Iter_Valid
    iter_test = Iter_Valid(test_df,max_user=1000)
    predicted = []
    def set_predict(df):
        predicted.append(df)
else:
    import riiideducation
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict

In [ ]:
# cumcount = sum([len(df) for df in predicted])
# count = 0
# pbar = tqdm(total=cumcount)
# previous_test_df = None
# for (current_test, current_prediction_df) in iter_test:
#     count+=1
#     if previous_test_df is not None:
#         answers = eval(current_test["prior_group_answers_correct"].iloc[0])
#         responses = eval(current_test["prior_group_responses"].iloc[0])
#         previous_test_df['answered_correctly'] = answers
#         previous_test_df['user_answer'] = responses
#         # your feature extraction and model training code here
#     previous_test_df = current_test.copy()
#     current_test = current_test[current_test.content_type_id == 0]
#     # your prediction code here
#     current_test['answered_correctly'] = model.predict(current_test[features])  # 0.5
#     set_predict(current_test.loc[:,['row_id', 'answered_correctly']])
#     pbar.update(len(current_test))
# print(f"count {count} {len(predicted)}")

In [ ]:
valid_df

In [ ]:
%%time

prior_test_df = None
for (test_df, sample_prediction_df) in iter_test:
    if prior_test_df is not None:
        prior_test_df[target] = eval(test_df['prior_group_answers_correct'].iloc[0])
        prior_test_df = prior_test_df[prior_test_df[target] != -1].reset_index(drop = True)
        
        user_ids = prior_test_df['user_id'].values
        content_ids = prior_test_df['content_id'].values
        targets = prior_test_df[target].values

        # for user_id, content_id, answered_correctly in prior_test_df[["user_id", "content_id", target]].values:
        for user_id, content_id, answered_correctly in zip(user_ids, content_ids, targets):
            user_sum_dict[user_id] += answered_correctly
            user_count_dict[user_id] += 1
            content_sum_dict[content_id] += answered_correctly
            content_count_dict[content_id] += 1

    prior_test_df = test_df.copy()
    
    test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
    test_df.drop(labels="part", axis=1, inplace=True)
    test_df.content_id = test_df.content_id.astype(int)
    
    test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(False).astype('bool')    
    user_sum = np.zeros(len(test_df), dtype = np.int32)
    user_count = np.zeros(len(test_df), dtype = np.int32)
    content_sum = np.zeros(len(test_df), dtype = np.int32)
    content_count = np.zeros(len(test_df), dtype = np.int32)
    
    for i, (user_id, content_id) in enumerate(zip(test_df['user_id'].values, test_df['content_id'].values)):
        user_sum[i] = user_sum_dict[user_id]
        user_count[i] = user_count_dict[user_id]
        content_sum[i] = content_sum_dict[content_id]
        content_count[i] = content_count_dict[content_id]

    test_df['user_correctness'] = user_sum / user_count
    test_df['content_count'] = content_count
    test_df['content_id'] = content_sum / content_count
    test_df[target] = model.predict_proba(test_df[features])[:,1]
    set_predict(test_df[['row_id', target]])